In [1]:
import pandas as pd
import logging
import csv
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import re
import time
from pyspark.sql import functions as F
from pyspark.sql.functions import length,col, explode, upper, to_date, date_sub, lag, coalesce, lit, array_sort, when, arrays_zip, size, date_format, explode_outer, from_json, concat, expr, array
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.utils import AnalysisException
from operator import itemgetter
import datetime, re, requests
from pyspark.sql import Window
from pyspark.sql.functions import concat, col, lit 
from pyspark.sql.functions import split
from pyspark.sql.functions import when
from pyspark.sql.functions import current_timestamp
from datetime import datetime

In [2]:
# # read data from S3 to DataFrame
# # input: S3
# # output df['ROW_ID', 'BOS_FILE_EXTRACT', 'file_path', 'COLUMN_DEF']
# def read_s3_to_df(day_path):
#
#     # parameter
#     i = 0 # flag for initialize dataframe
#
#     # boto3 client
#     s3 = boto3.client('s3')
#     conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
#
#     # read data file by file
#     for key in conn.list_objects(Bucket='bos-etl')['Contents']:
#         path_key = key['Key']
#         if path_key.endswith('cleansed'):
#             if day_path in path_key:
#                 file = s3.get_object(Bucket='bos-etl', Key=path_key)
#                 txt = (file['Body'].read().decode('latin1'))
#                 #st_re = txt.replace("", ",")
#                 st_re_newline = txt.replace("!!! EOS !!!", "\n")
#                 st_re_split = st_re_newline.split("\n")
#                 df = pd.DataFrame(st_re_split)
#                 df.index.name = 'ROW_ID'
#                 df.rename({0:'BOS_FILE_EXTRACT'},axis='columns',inplace=True)
#                 df["COLUMN_DEF"]=df['BOS_FILE_EXTRACT'].replace(regex=r"\.*",value="")
#                 rslt_dfRFT_temp = df[df['COLUMN_DEF'] =='PAX']
#                 # rslt_dfRFT_temp = df
#                 if ~rslt_dfRFT_temp.empty:
#                     # print(path_key)
#                     rslt_dfRFT_temp.insert(0,'file_path', path_key)
#                     if (i ==0):
#                         rslt_dfRFT = rslt_dfRFT_temp
#                         i = 1
#                     else:
#                         rslt_dfRFT = pd.concat([rslt_dfRFT,rslt_dfRFT_temp])
#     return rslt_dfRFT
#
#
# # read data as DataFrame
# #select day
# day_path = 'date=2023-06-17'
# # select current system day
# # day_path = time.strftime('%Y-%m-%d')
# rslt_dfRFT = read_s3_to_df(day_path)
#
# # test
# #filter content
# # check_list=['123','456']
#
# # for i in range():
# #       rslt_dfRFT['BOS_FILE_EXTRACT'].filter(like=check_list[i])

In [3]:
# #test write to s3 csv
# from io import StringIO
# bucket = 'bos-etl' # already created on S3
# csv_buffer = StringIO()
# rslt_dfRFT.to_csv(csv_buffer)
# s3_resource = boto3.resource('s3')
# s3_resource.Object(bucket, 'write_back/df.csv').put(Body=csv_buffer.getvalue())

In [4]:
# read csv
rslt_dfRFT = pd.read_csv("source/input/1_df_RFT_PAT_PAX.csv")

In [3]:
# import sys
# from awsglue.transforms import *
# from awsglue.utils import getResolvedOptions
# from pyspark.context import SparkContext
# from awsglue.context import GlueContext
# from awsglue.job import Job
#
# ## @params: [JOB_NAME]
# args = getResolvedOptions(sys.argv, ['JOB_NAME'])
#
# sc = SparkContext()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)
# job.init(args['JOB_NAME'], args)

In [19]:
# Pyspark Dataframe
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(rslt_dfRFT)

In [184]:
# # Split Raw Data
#
# def split_raw_data(sparkDF):
#     df1 = sparkDF.withColumn("split", F.split("BOS_FILE_EXTRACT", "<")).withColumn("ITI", F.element_at("split", 2)).withColumn("FAR", F.element_at("split", 3))\
#     .withColumn("FOP_AND_FAR2", F.element_at("split", 4)).withColumn("END", F.element_at("split", 5)).withColumn("CER", F.element_at("split", 6))\
#     .withColumn("EXC", F.element_at("split", 7))\
#     .withColumn("split2", F.split("BOS_FILE_EXTRACT", "<")).withColumn("DCI", F.element_at("split2", 1)).withColumn("SAL", F.element_at("split2", 2))\
#     .withColumn("EXS", F.element_at("split2", -1))\
#     .withColumn("split3", F.split("BOS_FILE_EXTRACT", "<")).withColumn("TAX2", F.element_at("split3", 2))\
#     .withColumn("split4", F.split("BOS_FILE_EXTRACT", "<")).withColumn("TAX1", F.element_at("split4", 2))
#
#     split_df = df1.select("COLUMN_DEF","DCI","SAL","TAX1","TAX2","ITI","FAR","FOP_AND_FAR2","END","CER","EXC","EXS",split(df1.TAX1, '<').alias('split_text'),split(df1.FOP_AND_FAR2,'N<').alias("Splittext3"),"file_path","TAX2",F.regexp_extract(df1.BOS_FILE_EXTRACT, '<.+',0).alias('RFT'))
#     split_df_type=split_df.selectExpr("column_def","DCI","SAL","concat(split_text[0],',',TAX2) as TAX","ITI","FAR", "Case when column_def=='PAT' then FOP_AND_FAR2 else Splittext3[1] end as FOP","Case when column_def=='PAT' then CER else END end as END_T","Case when column_def=='PAT' then EXC else CER end as CER_T","Case when column_def=='PAT' then '' else EXC end as EXC","Case when column_def=='PAT' then '' else CER end as EXS_T","file_path","TAX2","RFT")
#
#     return split_df_type
#
# split_df_type = split_raw_data(sparkDF)

In [29]:
# Split Raw Data
#  Start of repeat
#  END REPEAT
#  END of Data Set Indicator
# <   Carrriage Return
# DCI < SAL < TAX < ITI < FAR < FOP < END < CER < EXC < EXS <
#     < REF <

def split_raw_data(sparkDF):
    split_df_type = sparkDF.withColumn("split", F.split("BOS_FILE_EXTRACT", ""))\
                 .withColumn("DCI_SAL", F.element_at("split", 1))\
                 .withColumn("split2", F.split("DCI_SAL", "<"))\
                 .withColumn("DCI", F.element_at("split2", 1))\
                 .withColumn("SAL", F.element_at("split2", 2))\
                 .withColumn("TAX", F.element_at("split", 2))\
                 .withColumn("ITI", F.element_at("split", 3))\
                 .withColumn("FAR", F.element_at("split", 4))\
                 .withColumn("FOP", F.element_at("split", 5))\
                 .withColumn("END", F.element_at("split", 6))\
                 .withColumn("CER", F.element_at("split", 7))\
                 .withColumn("EXC", F.element_at("split", 8))\
                 .withColumn("EXS",  F.element_at("split", 9))\
                 .withColumn("RFT",  F.regexp_extract('BOS_FILE_EXTRACT', '<.+<',0))

    return split_df_type

split_df_type = split_raw_data(sparkDF)

In [30]:
split_df_type.select('column_def',"DCI","SAL","TAX","ITI","FAR","FOP","END","CER","EXC","EXS","RFT").head(3)

[Row(column_def='RFT', DCI='RFT\x8022521623\x8020230618\x8023061805093326\x80125\x802161454822\x804\x8017JUN23\x80\x80IAR\x80BA/D/ALLRECORDS/18JUN23\x80103\x80', SAL='\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80', TAX='', ITI=None, FAR=None, FOP=None, END=None, CER=None, EXC=None, EXS=None, RFT='<\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80<\x82\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<\x83'),
 Row(column_def='PAX', DCI='PAX\x8022521623\x8020230618\x8023061805093326\x80001\x807976703339\x800\x8017JUN23\x80\x80IAR\x80BA/D/ALLRECORDS/18JUN23\x80103\x80', SAL='\x80\x80\x80\x80\x80\x80FFVV\x80203.20\x80166.51\x80USD\x800.00\x8036.69\x801.00\x800.60\x80HNKNVK/AA\x80REDACTED SAL NAME\x80\x80\x80\x800011\x80/\x80\x80Y\x80\x80\x800\x80\x80\x8000\x80', TAX='\x8112

In [45]:
split_df_type.select('column_def','TAX').head(3)

[Row(column_def='RFT', TAX=''),
 Row(column_def='PAX', TAX='\x8112.49\x80US\x80<9.60\x80ZP\x80<5.60\x80AY\x80<9.00\x80XF\x80<\x82\x81ORF\x804.5\x80<DCA\x804.5\x80<\x82'),
 Row(column_def='PAT', TAX='\x812.20\x80YR\x80<16.00\x80CJ\x80<22.50\x80RN\x80<28.60\x80VV\x80<15.70\x80JD\x80<0.70\x80OG\x80<3.50\x80QV\x80<\x82\x81\x80\x80<\x82')]

In [32]:
# Common
# etl main fc
def etl_fc(df, fc, input, output):
    # fc
    fc = udf(fc, ArrayType(StringType()))

    # input
    fc_list = fc(*input)

    # output
    for i in range(len(output)):
        df = df.withColumn(output[i], fc_list[i])

    return df

# data validation fc
def dv_fc(name, msg, fc , *args):
    if name == 'check_len':
        print(*args)
        print(len(args))


In [82]:
# data validation fc
def dv_fc(rule_name, args_name, args, msg, paras ):

    #rules
    if rule_name == 'check_len':
        # ValueOutOfRange
        (args, msg) = (args, msg) if (len(str(x).split('.')[0]) < (paras[0] - paras[1])) & (len(str(x).split('.')[1]) < paras[1]) else ('NULL', msg + ' # ' + args_name +'OutOfRange: ' + str(args) + ' out of (' + str(paras[0]) + ',' + str(paras[1])  + ') # ')


    if rule_name == 'check_match':
        # Value1MismatchValue2
        msg = msg if str(args[0]) == str(args[1]) else msg + ' # ' + args_name[0] +'MisMatch' + args_name[1] + ': (' + args_name[0] + ' , ' + args_name[1]  + ') : (' + str(args[0]) + ' , '  + str(args[1]) + ') # '

    if rule_name == 'check_total':
        #ValueCheckTotalFail
        msg = msg if float(args[0]) == sum([float(x) for x in args[1]]) else  ' # ' + args_name[0] +'MisMatch' + args_name[1] + ': (' + args_name[0] + ' , ' + args_name[1]  + ') : (' + str(float(args[0])) + ' , '  + str(sum([float(x) for x in args[1]])) + ') # '

    return (args, msg)



# ExchangeRate = '12.345'
# (args, msg) = dv_fc(rule_name = 'check_len', args_name = 'ExchangeRate', args = ExchangeRate, msg = '', paras=(6, 5))
# print(msg)
# print(args)
# (args, msg) = dv_fc(rule_name = 'check_match', args_name = ('leg','fare'), args = (2,2), msg = '', paras = None)
# print(args)
# print(msg)
# (args, msg) = dv_fc(rule_name = 'check_total', args_name = ('leg','fare'), args = (7,[1,'2','3']), msg = '', paras = None)
# print(args)
# print(msg)

In [33]:
# Custom
# PAT/PAX/REF : [transaction_date, source, booking_channel, version_no, currency_code, ticket_type]

def fc_custom(data):

    # var
    transaction_date = current_timestamp() # <<File Generated Date>>
    source = 'BOS' # BOS
    booking_channel = 'WEB' # WEB
    version_no = 1
    currency_code = 'USD' # always “USD”

    ''' ticket_type :
         line starts with PAT → ticket type = TKTT
         line starts with PAX→ ticket type = EXCH-TKTT
         line starts with RFT → ticket type = RFND
         '''

    # insert var
    data = data.withColumn('transaction_date', transaction_date)\
               .withColumn('source', F.lit(source)) \
               .withColumn('booking_channel', F.lit(booking_channel))\
               .withColumn('version_no', F.lit(version_no))\
               .withColumn('ticket_type', when(col('column_def') == 'PAT', 'TKTT').\
                                          when(col('column_def') == 'PAX', 'EXCH-TKTT').\
                                          when(col('column_def') == 'RFT', 'RFND').\
                                          otherwise(''))\
               .withColumn('currency_code', F.lit(currency_code))

    return data

bos_df_csv = fc_custom(split_df_type)

In [34]:
bos_df_csv.select('column_def','transaction_date' , 'source', 'booking_channel', 'ticket_type','version_no').show()

+----------+--------------------+------+---------------+-----------+----------+
|column_def|    transaction_date|source|booking_channel|ticket_type|version_no|
+----------+--------------------+------+---------------+-----------+----------+
|       RFT|2023-07-30 22:51:...|   BOS|            WEB|       RFND|         1|
|       PAX|2023-07-30 22:51:...|   BOS|            WEB|  EXCH-TKTT|         1|
|       PAT|2023-07-30 22:51:...|   BOS|            WEB|       TKTT|         1|
+----------+--------------------+------+---------------+-----------+----------+



In [35]:
# DCI
# PAT/PAX/REF : ['agency_code', 'ticket_number', 'issue_date']
input  = ['DCI']
output = ['agency_code', 'ticket_number', 'issue_date']

def fc(DCI):

    # 1.split
    # print(DCI)
    DCI_split = DCI.split("")
    # print(DCI_split)

    # 2.var
    # BRE
    agency_code = DCI_split[1] # 2nd field from 1st group (VLNC-DCI)
    ticket_number = DCI_split[4] + DCI_split[5] # 5th field + 6th field from 1st group (BACN-DCI+ BDNR-DCI)
    issue_date = datetime.strptime(DCI_split[7],'%d%b%y').strftime("%Y-%m-%d") # 8th field from 1st group (DAIS-DCI). Field is reported as YYMMDD

    return [agency_code, ticket_number, issue_date]

bos_df_csv = etl_fc(bos_df_csv, fc, input, output)

In [36]:
bos_df_csv.select('column_def','agency_code','ticket_number','issue_date').show()

+----------+-----------+-------------+----------+
|column_def|agency_code|ticket_number|issue_date|
+----------+-----------+-------------+----------+
|       RFT|   22521623|1252161454822|2023-06-17|
|       PAX|   22521623|0017976703339|2023-06-17|
|       PAT|   22521623|0742100377105|2023-06-16|
+----------+-----------+-------------+----------+



In [51]:
# SAL
# PAT/PAX : ['pnr', 'tour_code', 'passenger_name', 'coupon_used', 'original_fare', 'fare_amount', 'exchange_rate', 'commission_amount', 'original_currency', 'tax_amount', 'total_amount', 'commission']

input  = ['column_def', 'SAL']
output = ['pnr', 'tour_code', 'passenger_name', 'coupon_used', 'original_fare', 'fare_amount', 'exchange_rate', 'commission_amount', 'original_currency', 'tax_amount', 'total_amount', 'commission']

def fc(column_def, SAL):

    # 1.split
    if column_def != 'RFT':
        # print(SAL)
        SAL_split = SAL.split("") if column_def != 'RFT' else None
        # print(SAL_split)

    # 2.var
    # BRE
    pnr = SAL_split[14] if column_def != 'RFT' else None # 15th field from 2nd group (PNRR-SAL). Note: it will be empty for RFND.
    tour_code = SAL_split[16] if column_def != 'RFT' else None # 17th field from 2nd group (TOUR-SAL). Note: it will be empty for RFND.
    passenger_name = SAL_split[15] if column_def != 'RFT' else None # 16th field from 2nd group (PXNM-SAL) for non-RFND.
    coupon_used = SAL_split[15] if column_def != 'RFT' else None # 7th field from 2nd group (CPUI-SAL) for non-RFND
    original_fare = SAL_split[8] if column_def != 'RFT' else None # 9th field from 2nd group
    original_currency = SAL_split[9] if column_def != 'RFT' else None# 10th field from 2nd group (CUOF-SAL)
    tax_amount = SAL_split[11] if column_def != 'RFT' else None # 12th field from 2nd group (TTAX-SAL)
    total_amount = SAL_split[7] if column_def != 'RFT' else None # 8th field from 2nd group (TDAM-SAL)

    fare_amount = None
    exchange_rate = None
    commission_amount = None
    commission = None
    if column_def != 'RFT':
        fare_amount = SAL_split[10] if SAL_split[10] != '0.00' else original_fare # 11th field from 2nd group (EQFR-SAL)… Note: if is 0.00, use the same as ORIGINAL_FARE

        exchange_rate = round(float(fare_amount) / float(original_fare),3) if original_fare != '0.00' else None # FARE_AMOUNT / ORIGINAL_FARE
        commission_amount = fare_amount # same as FARE_AMOUNT

        ''' COMMISSION:
        #JSON. Example: [{"type":"BASE","amount":4.26,"currency":"CAD","commissionRate":3.0}]
                       amount: 13th field from 2nd group (COAM-SAL)
                       commissionRate: 14th field from 2nd group (CORT-SAL)'''
        amount = SAL_split[12]
        commissionRate = SAL_split[13]
        type1 = 'BASE'
        currency = 'CAD'
        commission = '[{' + f'"type":"{type1}","amount":{amount},"currency":"{currency}","commissionRate":{commissionRate}' + '}]'

    return [pnr, tour_code, passenger_name, coupon_used, original_fare, fare_amount, exchange_rate, commission_amount, original_currency, tax_amount, total_amount, commission]

bos_df_csv = etl_fc(bos_df_csv, fc, input, output)

In [52]:
bos_df_csv.select('column_def','pnr','tour_code', 'passenger_name', 'original_fare', 'fare_amount','exchange_rate','commission_amount','original_currency','tax_amount','total_amount','commission').head(3)

[Row(column_def='RFT', pnr=None, tour_code=None, passenger_name=None, original_fare=None, fare_amount=None, exchange_rate=None, commission_amount=None, original_currency=None, tax_amount=None, total_amount=None, commission=None),
 Row(column_def='PAX', pnr='HNKNVK/AA', tour_code='', passenger_name='REDACTED SAL NAME', original_fare='166.51', fare_amount='166.51', exchange_rate='1.0', commission_amount='166.51', original_currency='USD', tax_amount='36.69', total_amount='203.20', commission='[{"type":"BASE","amount":1.00,"currency":"CAD","commissionRate":0.60}]'),
 Row(column_def='PAT', pnr='UOAGUL', tour_code='ITAFKL', passenger_name='REDACTED SAL NAME', original_fare='213.00', fare_amount='213.00', exchange_rate='1.0', commission_amount='213.00', original_currency='USD', tax_amount='89.20', total_amount='302.20', commission='[{"type":"BASE","amount":0.00,"currency":"CAD","commissionRate":0.00}]')]

In [53]:
bos_df_csv.select('column_def','TAX').head(3)

[Row(column_def='RFT', TAX=''),
 Row(column_def='PAX', TAX='\x8112.49\x80US\x80<9.60\x80ZP\x80<5.60\x80AY\x80<9.00\x80XF\x80<\x82\x81ORF\x804.5\x80<DCA\x804.5\x80<\x82'),
 Row(column_def='PAT', TAX='\x812.20\x80YR\x80<16.00\x80CJ\x80<22.50\x80RN\x80<28.60\x80VV\x80<15.70\x80JD\x80<0.70\x80OG\x80<3.50\x80QV\x80<\x82\x81\x80\x80<\x82')]

In [194]:
 # # Template
# # PAT/PAX : []
#
# input  = ['column_def', 'TAX']
# output = ['df']
#
# def fc(column_def, TAX):
#
#     # split
#     print(TAX_split)
#     TAX_split = TAX.split("")
#     # print(DCI_split)
#
#     # var
#     df = ''
#
#
#
#     return [df]
#
# bos_df_csv2 = etl_fc(bos_df_csv, fc, input, output)

In [58]:
# TAX
# PAT/PAX : ['tax']

input  = ['column_def', 'TAX', 'original_currency']
output = ['tax']

def fc(column_def, TAX, original_currency):

    # split
    if column_def != 'RFT':
        # print(TAX)
        TAX_split = TAX.split('<')[0].replace('','').split('<')
        # print(TAX_split)

    # var
    # BRE
    ''' TAX:
            Field will be <null> if is RFND
            Note: this group may have a loop
            JSON. Example: [{"type":"CA","amount":7.12,"currency":"CAD"},{"type":"YR","amount":16.00,"currency":"CAD"}]
            type: 2nd field of the loop from 3rd group (TMFT-TAX)
            amount: 1st field of the loop from 3rd group (TMFA-TAX)
            currency: 10th field from 2nd group (CUOF-SAL)'''
    tax = None
    if column_def != 'RFT':
        tax = ''
        for i in range(len(TAX_split)):
            # split
            element = TAX_split[i].split('')
            # print(element)

            # element
            type1 = element[1]
            amount = element[0]
            currency = original_currency

            # tax
            s = ',' if i != 0 else '['
            tax = tax + s + '{' + f'"type":"{type1}","amount":{amount},"currency":"{currency}"' + '}'

        tax = tax + ']'

    return [tax]

bos_df_csv = etl_fc(bos_df_csv, fc, input, output)

In [59]:
bos_df_csv.select('column_def', 'tax').head(3)

[Row(column_def='RFT', tax=None),
 Row(column_def='PAX', tax='[{"type":"US","amount":12.49,"currency":"USD"},{"type":"ZP","amount":9.60,"currency":"USD"},{"type":"AY","amount":5.60,"currency":"USD"},{"type":"XF","amount":9.00,"currency":"USD"}]'),
 Row(column_def='PAT', tax='[{"type":"YR","amount":2.20,"currency":"USD"},{"type":"CJ","amount":16.00,"currency":"USD"},{"type":"RN","amount":22.50,"currency":"USD"},{"type":"VV","amount":28.60,"currency":"USD"},{"type":"JD","amount":15.70,"currency":"USD"},{"type":"OG","amount":0.70,"currency":"USD"},{"type":"QV","amount":3.50,"currency":"USD"}]')]

In [62]:
# ITI
# PAT/PAX : ['legs']

input  = ['column_def', 'ITI', 'ticket_number', 'currency_code', 'original_currency', 'issue_date', 'FAR']
output = ['legs']

def fc(column_def, ITI, ticket_number, currency_code, original_currency, issue_date, FAR):

    # split
    # ITI
    if column_def != 'RFT' :
        # print(ITI)
        ITI_split = ITI.replace('','').replace('<','').split('<')
        # print(ITI_split)
        if (len(ITI_split)>1):
            # if two more than two ticket legs, direction from right -> left
            ITI_split.reverse()
    # FAR
        FAR_split = re.findall(r'[A-PR-Z]\d+\.\d+', FAR.split("END")[0]) # filter startswith 'Q'
        FAR_split = [x[1:] for x in FAR_split ] # filter start alpha
        if (len(FAR_split) > 1):
            # if two more than two ticket legs, direction from right -> left
            FAR_split.reverse()
        # toal_far
        fare_total = FAR_split[0]
        # leg far
        fare_legs = FAR_split[1:]

    # var
    '''LEGS:
        NOTE: If the ticket type is RFND, LEGS field needs to be empty

        Note: this info is provided in the ITI group. Each ticket may have 4 legs max. After the 5th leg, the ticket is considered as conjunction. In the file, if there is a loop, the ticket has a conjunction ticket.Example: 1st leg, 2nd leg, 3rd leg, 4th leg + loop + 5th leg…If the leg is empty, it means that the ticket stopped in the previous leg. Don’t load empty values in the Json.

        JSON. example: [{"departure":"FLR","destination":"YYZ","seatClass":"C","conjunction":"1111234567890","carrier":"AC","tripCode":"876","departureOn":"2022-12-30","designator":"","stopOver":"X","flyerCode":"","fare":259.25,"currency": "CAD","originalFare":259.25,"originalCurrency":"CAD"}]
        departure: from 4th group (ORAC-ITI) → Leg 1: 4th field | Leg 2: 13th field | Leg 3: 22nd field | Leg 4: 31st field
        destination: from 4th group (DSTC-ITI) → Leg 1: 5th field | Leg 2: 14th field | Leg 3: 23rd field | Leg 4: 32nd field
        seatClass: 4th group (CLSC-ITI) → Leg 1: 8th field | Leg 2: 17th field | Leg 3: 26th field | Leg 4: 35th field
        conjunction: 1st field from 4th group (CJNR-ITI). If is empty, use the same as TICKET_NUMBER

        if the ticket includes more than 4 legs: in ITI section, the 39th field (the field after 4th repeat’s designator) will give a new BDNR (10 digits), use the BACN from DCI + new BDNR as conjunction for the following legs

        carrier: from 4th group (CARR-ITI) → Leg 1: 6th field | Leg 2: 15th field | Leg 3: 24th field | Leg 4: 33rd field
        tripCode: 4th group (FTNR-ITI) → Leg 1: 7th field | Leg 2: 16th field | Leg 3: 25th field | Leg 4: 34th field

        departured on: 4th group (FTDA-ITI) → Leg 1: 9th field | Leg 2: 18th field | Leg 3: 27th field | Leg 4: 36th field. NOTE: you need to store this format in the databse: YYYY-MM-DD but the file has JAN01 for example. Use the same procedure/logic from CAT file loader in order to convert into date

        Check the logic from CAT (Java code) with Haibinhg and Santhosh because there are some tricks in the code but the logic is:
        File will come as JUL01 (they don’t report the year) → convert into 2023-07-01
        If the departure date is before the issue date, the year will be ISSUE_DATE +1 year. Example: issue date is 2023-07-01 and departure date is JUN01, then the departure date will be 2024-06-01
        If the departure date is after or equals to the issue date, the year will be the same as Issue Date. Example: issue date is 2023-07-01 and departure date is DEC01, then the departure date will be 2023-12-01
        If the departure date is after the issue date (but after december 31st), the year will be the same as Issue Date + 1 year. Example: issue date is 2023-07-01 and departure date is JAN01, then the departure date will be 2024-01-01
        designator: 4th group (FBTD-ITI) → Leg 1: 11th field | Leg 2: 20th field | Leg 3: 29th field | Leg 4: 38th field
        stopOver: from 4th group (STPO-ITI) → Leg 1: 3rd field | Leg 2: 12th field | Leg 3: 21st field | Leg 4: 30th field
        flyerCode: <empty>

        fare: you need to check the fare construction group and then apply the same logic as CAT loader (use the same rules applied on these stories: FTS-1518, FTS-1188 item 2, FTS-1188 and FTS-1502)
        currency: same as CURRENCY_CODE

        originalFare: you need to check the fare construction group and then apply the same logic as CAT loader (use the same rules applied on these stories: FTS-1518 and FTS-1188 item 2)
        originalCurrency: same as ORIGINAL_CURRENCY'''

    legs = None
    if column_def != 'RFT':
        legs = ''
        for i in range(len(ITI_split)):
            # split
            element = ITI_split[i].split('')

            # segment
            j = 30 # index for element
            k = 0 # index for fare_legs
            conjunction = ticket_number if len(element[0]) == 0 else (ticket_number[0:3] + element[0])
            currency = currency_code
            originalCurrency = original_currency
            while (j > 0):
                if len(element[j]) != 0:
                    # element
                    departure = element[j]
                    destination = element[j+1]
                    seatClass = element[j+4]
                    carrier = element[j+2]
                    tripCode = element[j+3]
                    designator = element[j+7]
                    stopOver = element[j-1]
                    flyerCode = ''

                    # fare
                    # far if 'X' != 0, take one from fare_leg
                    if (stopOver == 'X'):
                        fare = str('0.00')
                    else:
                        if (fare_legs == 0):
                            fare = 'NULL'
                        elif (k < len(fare_legs)):
                            fare = fare_legs[k]
                            k = k + 1
                        else:
                            fare = 'NULL'

                    originalFare = fare

                    # departureOn
                    # dep_on_date(issue_year-mm-dd)if dep_on_date(mm-dd) > issue_data(mm-dd) else dep_on_date((issue_year+1)-mm-dd))
                    dep_on_date = element[j+5]
                    issue_year = issue_date[0:4]
                    dep_day = datetime.strptime(dep_on_date + '2024','%d%b%Y').strftime("%m-%d") # 2024 is leap year to avoid '02-28', just for transformation, not use it afterward
                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)+ 1)
                    departureOn = str(pd.datetime.strptime(dep_on_date + dep_year,'%d%b%Y'))[0:10]
                    # departureOn = ''

                    # legs
                    s = '[' if ((i == len(ITI_split) - 1) and (j == 3)) else ','
                    legs = s + '{' + f'"departure":"{departure}","destination":"{destination}","seatClass":"{seatClass}","conjunction":"{conjunction}","carrier":"{carrier}","tripCode":"{tripCode}","departureOn":"{departureOn}","designator":"{designator}","stopOver":"{stopOver}","flyerCode":"{flyerCode}","fare":{fare},"currency":"{currency}","originalFare":{originalFare},"originalCurrency":"{originalCurrency}"' + '}' + legs

                j = j - 9

        legs = legs + ']'

    return [legs]

bos_df_csv = etl_fc(bos_df_csv, fc, input, output)

In [64]:
bos_df_csv.select('column_def','legs').head(3)

XORFDCAAA5508S08JUL323PSUAIZNN1DCAGRRAA5091S08JUL530PSUAIZNN1VV<
['\x80\x80X\x80ORF\x80DCA\x80AA\x805508\x80S\x8008JUL\x80323P\x80SUAIZNN1\x80\x80DCA\x80GRR\x80AA\x805091\x80S\x8008JUL\x80530P\x80SUAIZNN1\x80V\x80\x80\x80\x80\x80\x80\x80\x80\x80V\x80\x80\x80\x80\x80\x80\x80\x80\x80']
/var/folders/d9/871p0v1d2cn4c6y7bj_xghhh0000gn/T/ipykernel_2543/71694976.py:109: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
OAMSMADKL1701Q24JUN0930QH5UA5LG/XXOMADAMSKL1702E27JUN1305EH5UA5LGVV<
['\x80\x80O\x80AMS\x80MAD\x80KL\x801701\x80Q\x8024JUN\x800930\x80QH5UA5LG/XX\x80O\x80MAD\x80AMS\x80KL\x801702\x80E\x8027JUN\x801305\x80EH5UA5LG\x80V\x80\x80\x80\x80\x80\x80\x80\x80\x80V\x80\x80\x80\x80\x80\x80\x80\x80\x80']


[Row(column_def='RFT', legs=None),
 Row(column_def='PAX', legs='[{"departure":"ORF","destination":"DCA","seatClass":"S","conjunction":"0017976703339","carrier":"AA","tripCode":"5508","departureOn":"2023-07-08","designator":"SUAIZNN1","stopOver":"X","flyerCode":"","fare":0.00,"currency":"USD","originalFare":0.00,"originalCurrency":"USD"},{"departure":"DCA","destination":"GRR","seatClass":"S","conjunction":"0017976703339","carrier":"AA","tripCode":"5091","departureOn":"2023-07-08","designator":"SUAIZNN1","stopOver":"","flyerCode":"","fare":166.51,"currency":"USD","originalFare":166.51,"originalCurrency":"USD"}]'),
 Row(column_def='PAT', legs='[{"departure":"AMS","destination":"MAD","seatClass":"Q","conjunction":"0742100377105","carrier":"KL","tripCode":"1701","departureOn":"2023-06-24","designator":"QH5UA5LG/XX","stopOver":"O","flyerCode":"","fare":119.12,"currency":"USD","originalFare":119.12,"originalCurrency":"USD"},{"departure":"MAD","destination":"AMS","seatClass":"E","conjunction":

In [67]:
# FAR
# PAT/PAX : [fare_construction]

input  = ['column_def', 'FAR']
output = ['fare_construction']

def fc(column_def, FAR):

    # split
    if column_def != 'RFT':
        # print(FAR)
        FAR_split =FAR.replace('','').replace('<','').split('<') # filter the last one which is ''
        # print(FAR_split)

    # var
    '''FARE_CONSTRUCTION:
        Note: the FAR group might have a loop, that’s why we need to use sequence 1, sequence 2, etc.
        JSON. Example: [{"sequence":1,"content":"AX373911153791006*0626/ 122948"},{"sequence":2,"content":"YHZ PD YMQ54.56CAD54.56END"}]
        content: 1st field from 5th group (FRCA-FAR)'''
    fare_construction = None
    if column_def != 'RFT':
        fare_construction = ''
        for i in range(len(FAR_split)):
            element = FAR_split[i].split('')
            content = element[0]
            s = ',' if i != 0 else '['
            fare_construction = fare_construction + s + '{' + f'"sequence":{i+1},"content":"{content}"' +'}'

        fare_construction = fare_construction + ']'

    return [fare_construction]

bos_df_csv = etl_fc(bos_df_csv, fc, input, output)

In [68]:
bos_df_csv.select('fare_construction').head(3)

[Row(fare_construction=None),
 Row(fare_construction='[{"sequence":1,"content":"ORF AA X/WAS AA GRS166.51USD166.51END ZPORFDCA XFORF4.5DCA4.5"}]'),
 Row(fare_construction='[{"sequence":1,"content":"AMS KL MAD119.12KL AMS97.52NUC216.64END ROE0.907418XT22.50RN28.60VV15.70JD0.70OG3.50QV"}]')]

In [99]:
# FOP
# PAT/PAX : [payment]

input  = ['column_def', 'FOP', 'currency_code', 'EXC']
output = ['payment', 'first_payment_accountNumber']

def fc(column_def, FOP, currency_code, EXC):

    # split
    # FOP
    print(column_def)
    if column_def != 'RFT':
        # print(FOP)
        FOP_split = FOP.replace('','').replace('<','').split('<')
        # print(FOP_split)

    # EXC
    if column_def == 'PAX':
        EXC_split = EXC.split('<')
        EXC1_split = EXC_split[0].replace('','').split('<')# loop of NACN+NDNR+NCDT
        EXC3_split = EXC_split[2].replace('','').split('<') # loop of RCPU + PXNM

    # var
    '''PAYMENT:
        JSON. Example: [{"mode":"CC","type":"CCXX","amount":0.00,"accountNumber":"","approvalCode":"","invoiceNumber":"","currency":"CAD"}]
        mode: 1st field of the loop from 7th group (FPTP-FOP). Use only the first 2 chars
        type: 1st field of the loop from 7th group (FPTP-FOP)
        amount: 6th field of the loop from 7th group (FPAM-FOP)
        accountNumber: 2nd field of the loop from 7th group (FPAC-FOP)
        approvalCode: 5th field of the loop from 7th group (APLC-FOP)
        invoiceNumber: <empty>
        currency: same as CURRENCY_CODE

        for EXCH, the PAYMENT will be like the example below:
        Besides the regular FOP above, we need to add the EX info. Example: [{"mode":"EX","type":"EX","amount":0.00,"accountNumber":"451123456789001","approvalCode":"","invoiceNumber":"","currency":"CAD"},{"mode":"CC","type":"CCXX","amount":250.00,"accountNumber":"","approvalCode":"","invoiceNumber":"","currency":"CAD"}]
        accountNumber: 1st field + 2nd field + 3rd field + 19th field from 10th group (NACN-EXC+ NDNR-EXC + NCDT-EXC + RCPU-EXC). Note: RCPU is char and needs to be converted as number (use the same logic as “coupons field” from refund_legs.
        amount: 0.00'''

    payment = None
    if column_def != 'RFT':
        payment = ''
        currency = currency_code
        for i in range(len(FOP_split)):
            # split
            FOP_element = FOP_split[i].split("")
            # print(FOP_element)
            EXC1_element, EXC3_element = 'NULL', 'NULL'
            if column_def == 'PAX':
                EXC1_element = EXC1_split[0].split("")
                EXC3_element = EXC3_split[0].split("")

            # payment
            # element
            mode , type1, accountNumber = '', '', ''
            accountNumber = ''
            if column_def == 'PAT':
                mode = FOP_element[0][:2]
                type1 = FOP_element[0]
                accountNumber = FOP_element[1]

            if column_def == 'PAX':
                mode = 'EX'
                type1 = 'EX'
                accountNumber = EXC1_element[0] + EXC1_element[1] + EXC1_element[2] + EXC3_element[0]
                first_payment_accountNumber = accountNumber if i == 0

            amount = FOP_element[5]
            approvalCode = FOP_element[5]
            invoiceNumber = ''

            s = ',' if i != 0 else '['
            payment = payment + s + '{' + f'"mode":"{mode}","type":"{type1}","amount":{amount},"accountNumber":"{accountNumber}","approvalCode":"{approvalCode}","invoiceNumber":"{invoiceNumber}","currency":"{currency}"' +'}'

        payment = payment + ']'

    return [payment, first_payment_accountNumber]

bos_df_csv = etl_fc(bos_df_csv, fc, input, output)

In [100]:
bos_df_csv.select('column_def', 'payment').head(3)

RFT
PAX
CACASHN 203.200.001.000.60N<CACASHN25.000.001.000.60E<
['CA\x80CASH\x80N \x80\x80\x80203.20\x80\x80\x800.00\x80\x801.00\x800.60\x80N\x80', 'CA\x80CASH\x80N\x80\x80\x8025.00\x80\x80\x800.00\x80\x801.00\x800.60\x80E\x80']
PAT
CACASHN302.200.000.000.00N<
['CA\x80CASH\x80N\x80\x80\x80302.20\x80\x80\x800.00\x80\x800.00\x800.00\x80N\x80']


[Row(column_def='RFT', payment=None),
 Row(column_def='PAX', payment='[{"mode":"EX","type":"EX","amount":203.20,"accountNumber":"00179767033390F","approvalCode":"203.20","invoiceNumber":"","currency":"USD"},{"mode":"EX","type":"EX","amount":25.00,"accountNumber":"00179767033390F","approvalCode":"25.00","invoiceNumber":"","currency":"USD"}]'),
 Row(column_def='PAT', payment='[{"mode":"CA","type":"CA","amount":302.20,"accountNumber":"CASH","approvalCode":"302.20","invoiceNumber":"","currency":"USD"}]')]

In [111]:
# RFT
# RFT : [refund_legs]

input  = ['column_def', 'RFT', 'ticket_number', 'first_payment_accountNumber']
output = ['refund_legs', 'org_ticket_no']

def fc(column_def, RFT, ticket_number, first_payment_accountNumber):

    # split
    if column_def == 'RFT':
        # print(RFT)
        RFT_split = RFT.split('')
        # print(RFT_split)
        RFT_split_1 = re.search('.+<',RFT).group().replace('','').replace('<','').split('<')
        # print(RFT_split_1)
        # loop RACN + RDNR + RCPN
        RFT_split_2 = RFT_split[1] # ODOI
        # print(RFT_split_1)
        # print(RFT_split_2)

    # var
    # BRE
    # refund_legs
    '''REFUND_LEGS:
        note: to be populated if ticket_type is RFND only
        JSON. example: [{"sequence":1,"ticketNumber":"0011259634355","coupons":"1000","issueDate":"2022-05-03"}]
        ticketNumber: 17th field + 18th fied from 2nd group (RACN-REF+ RDNR-REF). Note: it may have a loop here, so you need to use the sequence 1, sequence 2, etc in the JSON
        issueDate: 20th fied from 2nd group (ODOI-REF).
        coupons: 19th fied from 2nd group (RCPN-REF). Note: we receive this field as chars. You need to convert into numbers, use the same logic as CAT Loader (for example, the field comes as RR, you need to convert into “1200”. Or might come as VRRV, for example, you need to convert into “0230”. You need to apply the number/sequence only for the letter R; to the other letters (or blank) you need to put “0”)'''

    # element
    refund_legs = None
    if column_def == 'RFT':
        issueDate = RFT_split_2.split("")[0]
        for i in range(len(RFT_split_1)):
            element = RFT_split_1[i].split("")

            # element
            ticketNumber = element[0] + element[1]
            coupons = element[2]
            first_refund_legs_ticketNumber = ticketNumber if i == 0 else 'NULL'

            # refund_legs
            refund_legs = ''
            s = ',' if i != 0 else '['
            refund_legs = refund_legs + s + '{' + f'"sequence":{str(i+1)},"ticketNumber":"{ticketNumber}","coupons":"{coupons}","issueDate":"{issueDate}"' +'}'

        refund_legs = refund_legs + ']'

    # org_ticket_no
    '''ORG_TICKET_NO:
            TKTT → same as TICKET_NUMBER
            EXCH → same as 1st accountNumber from PAYMENT (with mode = “EX” and only the first 13 chars only)
            RFND → same as 1st ticketNumber from REFUND_LEGS'''
    if column_def == 'PAT':
        org_ticket_no = ticket_number
    elif column_def == 'PAX':
        org_ticket_no = first_payment_accountNumber
    elif column_def == 'RFT':
        org_ticket_no = first_refund_legs_ticketNumber
    else:
        org_ticket_no = 'NULL'


    return [refund_legs, org_ticket_no]

bos_df_csv2 = etl_fc(bos_df_csv, fc, input, output)

In [117]:
s = '[{"mode":"EX","type":"EX","amount":203.20,"accountNumber":"00179767033390F","approvalCode":"203.20","invoiceNumber":"","currency":"USD"},{"mode":"EX","type":"EX","amount":25.00,"accountNumber":"00179767033390F","approvalCode":"25.00","invoiceNumber":"","currency":"USD"}]'

re.findall(r'"accountNumber":".+{15}"',s)

error: multiple repeat at position 19

In [110]:
bos_df_csv2.select('column_def','refund_legs').head(3)

<953.650.000.00358.00595.650.000.000.000.0012521614548221234<REDACTED REF NAMEF-953.650.000.00-953.65CACASH<
['<\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80<', '\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<\x83']
['125\x802161454822\x801234\x80']
['125\x802161454822\x801234\x80']
REDACTED REF NAMEF-953.650.000.00-953.65CACASH<


[Row(column_def='RFT', refund_legs='[{"sequence":1,"ticketNumber":"1252161454822","coupons":"1234","issueDate":""}]'),
 Row(column_def='PAX', refund_legs=None),
 Row(column_def='PAT', refund_legs=None)]

In [ ]:
s = '12521614548221234<'
s.split('<')

In [ ]:
# FTS - 2610  Commision: "CAD" - > "USD"
# input: Dataframe
# output: ['Transaction_Date','Source','Booking_channel','DCI_split','DCI','Agency_code',]

def split_col(split_df_type):
    data=split_df_type.withColumn("Transaction_Date",current_timestamp()).withColumn("Source",F.lit('BOS')).withColumn("Booking_channel",F.lit('WEB')).withColumn("DCI_split", F.split("DCI", "")).withColumn("Agency_code", F.element_at("DCI_split", 2)).withColumn("Ticket_No", concat(F.element_at("DCI_split", 5),F.element_at("DCI_split", 6))).withColumn("Issue_Date", F.to_date(F.element_at("DCI_split", 8),"ddMMMyy"))\
    .withColumn("SAL_split", F.split("SAL", "")).withColumn("PNR", F.element_at("SAL_split", 15)).withColumn("Tour_code", F.regexp_replace(F.element_at("SAL_split", 17),'','')).withColumn("Passenger", F.element_at("SAL_split", 16)).withColumn("Coupon_used", F.element_at("SAL_split", 7)).withColumn("Original_Fare", F.element_at("SAL_split", 9)).withColumn("Fare", F.element_at("SAL_split", 11)).withColumn("Org_currency", F.element_at("SAL_split", 10)).withColumn("Tax_Amt", F.element_at("SAL_split", 12)).withColumn("Total_amt", F.element_at("SAL_split", 8))\
    .withColumn("ITI_split", F.split("ITI", "<"))\
    .withColumn("Leg2", F.element_at("ITI_split", 13)).withColumn("Leg3", F.element_at("ITI_split", 22)).withColumn("Leg4", F.element_at("ITI_split", 31))\
    .withColumn("FOP_split", F.split("FOP_T", ""))\
    .withColumn("TAX_split", F.split("tax", "<"))\
    .withColumn("Commission", concat(F.lit('[{"type":"BASE","amount":'),F.element_at("SAL_split", 13),f.lit(',"currency":"USD","commissionRate":'),F.element_at("SAL_split", 14),f.lit('}]')))\
    .withColumn("Fare_split", F.split("FAR", "<")).withColumn("FOP_split", F.split("FOP_T", "<")).withColumn("SAL2_split", F.split("TAX2", "")).withColumn("Passenger_rfnd", F.element_at("SAL2_split", 2))

    return data

bos_df = split_col(split_df_type)

In [ ]:
# # df2.write.options
# split_df_type.coalesce(1).write.format('csv').option('header','true').mode("overwrite").save("s3://bos-etl/write_back/")

In [57]:
### FTS - 2607
bos_df_csv=bos_df.selectExpr("ITI_split","column_def","Transaction_Date","Agency_code","'06.07_DCALLRECORDSAM' as Filename","Source","Booking_channel","case when column_def=='PAX' then 'EXCH-TKTT' when column_def=='PAT' then 'TKTT' else 'RFND' end as ticket_type", "1 as version_no","Ticket_No","Issue_date","PNR","Tour_code","case when column_def=='RFT' then passenger_rfnd else Passenger end as Passenger_name","1 as Passenger_Count","Coupon_used","'USA' AS Country_code","Original_Fare","case when Fare==0 then original_fare else fare end as Fare_amt","Org_currency","round((case when Fare==0 then original_fare else fare end)/Original_Fare,3) as exch_rate","case when Fare==0 then original_fare else fare end as comm_amt","Tax_Amt","Total_amt","'USD' as curr_code","ticket_no as org_ticket_no","Commission","TAX_split","FOP_split","Fare_split","file_path","REF_LG","SAL2_split","EXC_T")

In [41]:
bos_df.head(1)

[Row(column_def='RFT', DCI='RFT\x8022521623\x8020230618\x8023061805093326\x80125\x802161454822\x804\x8017JUN23\x80\x80IAR\x80BA/D/ALLRECORDS/18JUN23\x80103\x80', SAL='\x80\x80\x80\x80\x80953.65\x800.00\x800.00\x80358.00\x80595.65\x800.00\x800.00\x800.00\x800.00\x80\x80\x80\x81125\x802161454822\x801234\x80', TAX=None, ITI=None, FAR=None, FOP_T=None, END_T=None, CER_T=None, EXC_T=None, EXS_T=None, file_path='data/date=2023-06-17/DCALLRECORDSAM.pgp.asc_3e5155823ea22a963da362086c693cf3_cleansed', TAX2='\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<\x83', REF_LG='125\x802161454822\x801234\x80<\x82\x80REDACTED REF NAME\x80F\x80-953.65\x800.00\x800.00\x80\x80-953.65\x80CA\x80CASH\x80<\x83', Transaction_Date=datetime.datetime(2023, 7, 28, 16, 37, 42, 198614), Source='BOS', Booking_channel='WEB', DCI_split=['RFT', '22521623', '20230618', '23061805093326', '125', '2161454822', '4', '17JUN23', '', 'IAR', 'BA/D/ALLRECORDS/18JUN23', '103', ''], Agency_code='

In [46]:
# BOS - 116 RFT
def fc_Ref(Fare_amt, column_def, Original_Fare, Tax_Amt, Org_currency,Total_amt):
    if column_def == 'RFT':
        #RFT fare_amount commission_amount: 9th field from REF * (-1)
        Fare_amt = float(Original_Fare) * -1

        #RFT tax_amount: 10th field from REF * (-1)
        Tax_Amt = float(Org_currency) * -1

        #RFT total_amount: 8th field from REF * (-1)
        Total_amt = float(Total_amt) * -1

    return [Fare_amt, Tax_Amt, Total_amt]

fc_Ref = udf(fc_Ref, ArrayType(StringType()))
fc_Ref_list = fc_Ref('Fare_amt', 'column_def', 'Original_Fare','Tax_Amt','Org_currency','Total_amt')
bos_df_csv = bos_df_csv.withColumn('Fare_amt', fc_Ref_list[0])\
                        .withColumn('Tax_Amt', fc_Ref_list[1])\
                        .withColumn('Total_amt', fc_Ref_list[2])

In [47]:
bos_df_csv.select('Fare_amt', 'Tax_Amt', 'Total_amt').show()

+--------+-------+---------+
|Fare_amt|Tax_Amt|Total_amt|
+--------+-------+---------+
|  -358.0|-595.65|     -0.0|
|  166.51|  36.69|   203.20|
|  213.00|  89.20|   302.20|
+--------+-------+---------+



In [58]:
# FTS - 2609
def fc_Fare(li, column_def):
    fare_v = ''
    fare = ''
    fare_end_v = ''
    if column_def != 'RFT':
        fare_v = ''
        if li is not None:
            # FTS - 2609
            fare_end_v = ''
            fare_v = '['
            i = 0
            # print (fare_v)
            for ii in li:
                i = i + 1
                ii_split = ii.split("")

                if i > 1:
                    fare_v = fare_v + ',{"sequence":' + str(i) + ',"content":"' + ii_split[0] + '"}'
                    fare = fare + ',' + ii_split[0]
                else:
                    fare_v = fare_v + '{"sequence":' + str(i) + ',"content":"' + ii_split[0] + '"}'
                    fare = ii_split[0]
            fare_v = fare_v + ']'
        # FTS - 2609 spilt by "END" + " "
        # fare_split = fare.split(" ")
        fare_split = fare.split("END" + " ")[0].split(" ")
        # print(fare_split)

        for element in fare_split:
            # FTS - 2609 add ~(element.startswith("Q") & element[1].isdigit())
            # if element.endswith("END"):
            if len(element) > 1:
                if (False if (element.startswith("Q") & element[1].isdigit()) else True):
                    # FTS - 2609 list -> str
                    fare_end_v = fare_end_v + element

    return [fare_v, fare, fare_end_v]

fc_Fare = udf(fc_Fare, ArrayType(StringType()))
fc_Fare_list = fc_Fare('Fare_split', 'column_def')
bos_df_csv = bos_df_csv.withColumn('Fare_Cons', fc_Fare_list[0])\
    .withColumn("Fare", fc_Fare_list[1]).withColumn("Fare_end",fc_Fare_list[2]).withColumn("Fare_split", F.split("Fare", " "))


In [49]:
bos_df_csv.select('Fare_Cons', "Fare_end", "Fare_split").show()

+--------------------+--------------------+--------------------+
|           Fare_Cons|            Fare_end|          Fare_split|
+--------------------+--------------------+--------------------+
|                    |                    |                  []|
|[{"sequence":1,"c...|ORFAAX/WASAAGRR16...|[ORF, AA, X/WAS, ...|
|[{"sequence":1,"c...|AMSKLMAD119.12KLA...|[AMS, KL, MAD119....|
+--------------------+--------------------+--------------------+



In [59]:
# FTS - 2607
# tax
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def fc(li, column_def):
    tax_v = ''
    if column_def != 'RFT':
        tax_v =''
        if li is not None:
            #print(li)
            tax_v ='['
            i=0
            for ii in li:
                ii_split = ii.split(",")
                # FTS 2607 Bug 2 & 3 : filter string which not start with digit
                ii_split = [x for x in ii_split if x[0].isdigit()]
                #print(ii_split)
                for i1 in ii_split:
                    i=i+1
                    i1_split = i1.split("")
                    #print(i1_split)
                    if i>1:
                        # FTS-2607 Bug 1
                        tax_v = tax_v+ ',{"type":"'+ i1_split[1] +'","amount":'+i1_split[0] +',"currency":"' +  "Org_currency" + '"}'
                    else:
                        # FTS-2607 Bug 1
                        tax_v = tax_v+ '{"type":"'+ i1_split[1] +'","amount":'+i1_split[0] +',"currency":"' +   "Org_currency" + '"}'
            tax_v=tax_v+']'
    return tax_v

fc = udf(fc, StringType())
bos_df_csv = bos_df_csv.withColumn('Tax_v', fc('TAX_split', 'column_def'))
bos_df_csv=bos_df_csv.selectExpr("ITI_split","Fare_end","column_def","Transaction_Date","Agency_code","Filename","Source","Booking_channel","ticket_type", "version_no","Ticket_No","Issue_date","PNR","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Original_Fare","Fare_amt","Org_currency","exch_rate","comm_amt","Tax_Amt","Total_amt","curr_code","org_ticket_no","Commission","Fare_Cons","case when column_def !='RFT' then replace(Tax_v,'Org_currency',Org_currency) end as Tax",'FOP_split',"file_path","REF_LG","SAL2_split","EXC_T")

In [84]:
# FTS - 2606
# PAT - FOP : payment
# PAX - EXC : payment
def fc_FOP(FOP_split, column_def):
    fop_v =''
    if column_def == 'PAT' :
        if FOP_split is not None:
            fop_v =''
            i=0
            for ii in FOP_split:
                ii_split = ii.split(",")
                for i1 in ii_split:
                    i1_split = i1.split("") #FOP
                    mode = i1_split[0][:2] # 1st field of the loop from 7th group (FPTP-FOP). Use only the first 2 chars
                    type1 = i1_split[0] # 1st field of the loop from 7th group (FPTP-FOP)
                    amount = i1_split[5] # 6th field of the loop from 7th group (FPAM-FOP)
                    accountNumber = i1_split[1] # 2nd field of the loop from 7th group (FPAC-FOP)
                    approvalCode = i1_split[4] # 5th field of the loop from 7th group (APLC-FOP)
                    invoiceNumber = '' # <empty>
                    currency = "USD" # same as CURRENCY_CODE “USD”

                    s = ',' if i != 0 else '['
                    fop_v = fop_v + s + '{'  + f'"mode":"{mode}","type":"{type1}","amount":{amount},"accountNumber":"{accountNumber}","approvalCode":"{approvalCode}","invoiceNumber":"{invoiceNumber}","currency":"{currency}"' + '}'

                    i=i+1

            fop_v = fop_v+']'

    return [fop_v]

fc_FOP = udf(fc_FOP, ArrayType(StringType()))
fc_FOP_list = fc_FOP('FOP_split', "column_def")
bos_df_csv2 = bos_df_csv.withColumn('FOP', fc_FOP_list[0])
bos_df_csv2=bos_df_csv2.selectExpr("ITI_split","Fare_end","column_def","Transaction_Date","Agency_code","Filename","Source","Booking_channel","ticket_type", "version_no","Ticket_No","Issue_date","PNR","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Original_Fare","Fare_amt","Org_currency","exch_rate","comm_amt","Tax_Amt","Total_amt","curr_code","org_ticket_no","Commission","Fare_Cons","Tax","FOP","file_path","REF_LG","SAL2_split")

In [85]:
bos_df_csv2.select('Fop','column_def').head(3)

[Row(Fop='', column_def='RFT'),
 Row(Fop='', column_def='PAX'),
 Row(Fop='[{"mode":"CA","type":"CA","amount":302.20,"accountNumber":"CASH","approvalCode":"","invoiceNumber":"","currency":"USD"}]', column_def='PAT')]

In [86]:
from datetime import datetime
# FTS - 2608 rewrtie function
# FTS - 2608 "currency": "CAD" -> "currency": "USD"
def fc_LEG(li, fare_row, ticket_no, issue_date, column_def):
    leg_v =''
    attributes = ''
    if column_def != 'RFT':
        if re.match(r'^\d{4}-\d{2}-\d{2}$',str(issue_date)):
            issue_year = issue_date.year
            issue_date = issue_date.strftime('%Y-%m-%d')
            leap_year = '2024' # not use this value just avoid error from strptime
            if li is not None:
                #print(li)
                leg_v ='[' # segment 1
                leg_v_1 = '[' #segment 2
                # i=0
                # FTS - 2608
                i = 0
                j = 1 # index of fare , max len 2
                fareval = []
                if (len(li) > 1): # if two segment, reverse, direction right to left
                    li.reverse()
                    i = 3
                for ii in li:
                    ii_split = ii.split(",")
                    #print(ii_split)
                    for i1 in ii_split:
                        fareval = re.findall(r'\d+.\d+', str(fare_row))
                        if len(fareval)==0:
                            total_fareval = '0.00'
                            fareval=['0.00','0.00','0.00','0.00']
                        else:
                            total_fareval = fareval[-1]
                            fareval = fareval[:-1]
                        #print(fareval)
                        i = i - 1 # reverse
                        i1_split = i1.split("")
                        if len(i1_split[0])!=0:
                            conj= ticket_no[0:3] + i1_split[0]
                        else:
                            conj='CONJ'

                        if i>1:
                           if len(i1_split[30])!=0:
                                if len(i1_split[35])!=0:
                                    dep_day = datetime.strptime(i1_split[35] +  leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(   issue_year)    + 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[35]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[29]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1
                                leg_v_1 = ',{"departure":"'+ i1_split[30] +'","destination":"'+i1_split[31]         +'","seatClass":"' +  i1_split[34] +'","conjunction":"'+  conj      +'","carrier":"'+i1_split[32]+'","tripCode":"'+i1_split[    33]+'","departureOn":"'+dep_on+'","designator":"'+i1_split[   37]+'","stopOver":"'+i1_split[   29]+'","flyerCode":"","fare":'+ fare +    ',"currency": "USD","originalFare":'+ fare     +',"originalCurrency":"'+'org_currency'+'"}'

                           if len(i1_split[21])!=0:
                                if len(i1_split[26])!=0:
                                    dep_day = datetime.strptime(i1_split[26] +  leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(   issue_year)    + 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[26]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[20]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1
                                leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                                leg_v_1 = ',{"departure":"'+ i1_split[21] +'","destination":"'+i1_split[22]         +'","seatClass":"' + i1_split[25] +'","conjunction":"'+  conj       +'","carrier":"'+i1_split[23]+'","tripCode":"'+i1_split[    24]+'","departureOn":"'+dep_on+'","designator":"'+i1_split[  28]+'","stopOver":"'+i1_split[    20]+'","flyerCode":"","fare":'+ fare    +',"currency": "USD","originalFare":'+ fare        +',"originalCurrency":"'+'org_currency'+'"}' + leg_v_1

                           if len(i1_split[12])!=0:
                                if len(i1_split[17])!=0:
                                    dep_day = datetime.strptime(i1_split[17] +  leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(   issue_year)    + 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[17]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[11]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1
                                leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                                leg_v_1 = ',{"departure":"'+ i1_split[12] +'","destination":"'+i1_split[13]         +'","seatClass":"' + i1_split[16] +'","conjunction":"'+  conj       +'","carrier":"'+i1_split[14]+'","tripCode":"'+i1_split[    15]+'","departureOn":"'+ dep_on+'","designator":"'+i1_split[  19]+'","stopOver":"'+i1_split[    11]+'","flyerCode":"","fare":'+ fare    +',"currency": "USD","originalFare":'+ fare        +',"originalCurrency":"'+'org_currency'+'"}' + leg_v_1

                           if len(i1_split[3])!=0:
                                if len(i1_split[8])!=0:
                                    dep_day = datetime.strptime(i1_split[8] +   leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(   issue_year)    + 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[8]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[2]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1

                                leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                                leg_v_1 = ',{"departure":"'+ i1_split[3] +'","destination":"'+i1_split[4]       +'","seatClass":"' + i1_split[7] +'","conjunction":"'+  conj    +'","carrier":"'+i1_split[5]+'","tripCode":"'+i1_split[6]+'","departureOn":"'+dep_on+'","designator":"'+i1_split[  10]+'","stopOver":"'+i1_split[ 2]+'","flyerCode":"","fare":'+ fare +',"currency":     "USD","originalFare":'+ fare  +',"originalCurrency":"'+'org_currency'+'"}' +    leg_v_1

                        else:
                            if len(i1_split[30])!=0:
                                if len(i1_split[35])!=0:
                                    dep_day = datetime.strptime(i1_split[35] +  leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)+ 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[35]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[29]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1

                                leg_v = ',{"departure":"'+ i1_split[30] +'","destination":"'+i1_split[31]       +'","seatClass":"' + i1_split[34] +'","conjunction":"'+  conj       +'","carrier":"'+i1_split[32]+'","tripCode":"'+i1_split[33]+'","departureOn":"'+dep_on+'","designator":"'+i1_split[  37]+'","stopOver":"'+i1_split[  29]+'","flyerCode":"","fare":'+ fare +    ',"currency": "USD","originalFare":'+ fare         +',"originalCurrency":"'+'org_currency'+'"}'

                            if len(i1_split[21])!=0:
                                if len(i1_split[26])!=0:
                                    dep_day = datetime.strptime(i1_split[26] +  leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)    + 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[26]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[20]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1
                                leg_v = '' if leg_v == '[' else leg_v
                                leg_v =  ',{"departure":"'+ i1_split[21] +'","destination":"'+i1_split[22]      +'","seatClass":"' + i1_split[25] +'","conjunction":"'+  conj       +'","carrier":"'+i1_split[23]+'","tripCode":"'+i1_split[24]+'","departureOn":"'+dep_on+'","designator":"'+i1_split[    28]+'","stopOver":"'+i1_split[ 20]+'","flyerCode":"","fare":'+ fare     +',"currency": "USD","originalFare":'+ fare         +',"originalCurrency":"'+'org_currency'+'"}' + leg_v

                            if len(i1_split[12])!=0:
                                if len(i1_split[17])!=0:
                                    dep_day = datetime.strptime(i1_split[17] +  leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(issue_year)+ 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[17]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[11]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1
                                leg_v = '' if leg_v == '[' else leg_v
                                leg_v =   ',{"departure":"'+ i1_split[12] +'","destination":"'+i1_split[13]         +'","seatClass":"' + i1_split[16] +'","conjunction":"'+  conj       +'","carrier":"'+i1_split[14]+'","tripCode":"'+i1_split[    15]+'","departureOn":"'+dep_on+'","designator":"'+i1_split[  19]+'","stopOver":"'+i1_split[    11]+'","flyerCode":"","fare":'+ fare    +',"currency": "USD","originalFare":'+ fare        +',"originalCurrency":"'+'org_currency'+'"}' + leg_v

                            if len(i1_split[3])!=0:
                                #print(conj)
                                if len(i1_split[8])!=0:
                                    dep_day = datetime.strptime(i1_split[8] +   leap_year,'%d%b%Y').strftime("%m-%d")
                                    dep_year = str(issue_year) if dep_day >= issue_date[5:] else str(int(   issue_year)    + 1)
                                    dep_on = str(pd.datetime.strptime(i1_split[8]+ dep_year,'%d%b%Y'))[0:10]
                                else:
                                    dep_on= 'NULL'
                                if i1_split[2]=='X':
                                    fare = str('0.00')
                                else:
                                    if (len(fareval) == 0):
                                        fare ='NULL'
                                    elif (j <= len(fareval)):
                                        fare = fareval[-j]
                                        j = j + 1
                                    else:
                                        fare = 'NULL'
                                        j = j + 1
                                leg_v = '' if leg_v == '[' else leg_v
                                leg_v = '[{"departure":"'+ i1_split[3] +'","destination":"'+i1_split[4]         +'","seatClass":"' + i1_split[7] +'","conjunction":"'+conj+'","carrier":"'+i1_split[5]+'","tripCode":"'+i1_split[6]+'","departureOn":"'+dep_on+'","designator":"'+i1_split[  10]+'","stopOver":"'+i1_split[   2]+'","flyerCode":"","fare":'+ fare  +',"currency": "USD","originalFare":'+ fare      +',"originalCurrency":"'+'org_currency'+'"}' + leg_v

                            # segment 1 + segment2
                            leg_v_1 = '' if leg_v_1 == '[' else leg_v_1
                            leg_v = leg_v + leg_v_1
                        #print(pd.datetime.strptime(i1_split[8]+ str(datetime.now().year),'%d%b%Y'))
                leg_v=leg_v+']'

                # attributes: len(leg) > len(fare)
                attributes = '' if j == len(fareval) + 1 else ' # ' + 'FareLegMismatch: leg: ' + str(j-1) +  ' does not match fare: ' + str(len(fareval)) + ' # '

                # atributes: total_fareval <> leg: fareval
                leg_fer_sum = '%.2f'%sum([float(x[7:]) for x in re.findall(r'\"fare\"\:\d+\.\d+', leg_v)])
                attributes = attributes if total_fareval == leg_fer_sum else '#' + 'FareAmountMismatch: Leg sum amount: ' + str(leg_fer_sum) + ' does not match fare amount: ' + str(total_fareval)  + ' # '

        else:
            # issue_date is not datatime
            attributes = attributes + '#' + 'Issue_dateNotDatetime: issue_date: ' +  str(issue_date)  + ' # '

    return [leg_v, attributes]

fc_LEG = udf(fc_LEG, ArrayType(StringType()))
fc_LEG_list = fc_LEG('ITI_split','Fare_end','Ticket_no','Issue_date','column_def')
bos_df_csv = bos_df_csv.withColumn('Leg_v1', fc_LEG_list[0]).withColumn('attributes', fc_LEG_list[1])
bos_df_csv=bos_df_csv.selectExpr("FOP","column_def","Transaction_Date","Agency_code","Filename","Source","Booking_channel","ticket_type", "version_no","Ticket_No","Issue_date","PNR","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Original_Fare","Fare_amt","Org_currency","exch_rate","comm_amt","Tax_Amt","Total_amt","curr_code","org_ticket_no","Commission","Fare_Cons","Tax","case when column_def !='RFT' then replace(replace(Leg_v1,'CONJ',Ticket_no),'org_currency',Org_currency) end as Legs","attributes","file_path","REF_LG","SAL2_split")

In [88]:
bos_df_csv.select('Legs').show()

+----+
|Legs|
+----+
|null|
+----+



In [89]:
bos_df_csv2 = bos_df_csv

In [92]:
# Refund_Legs
# ONLY for RFT
def fc_refund_legs (REF_LG, column_def,SAL2_split, org_ticket_no, Coupon_used):
    refund_legs = ''
    if column_def == 'RFT':
        REF_LG_split = REF_LG.split('')[0].split('<')[:-1]
        org_date = SAL2_split[0]
        i = 1 # flag if it is the first & sequence
        for lg_split in REF_LG_split:
            s = ',' if i != 0 else '['
            lg_split_split = lg_split.split('')
            ticketNumber = lg_split_split[0] + lg_split_split[1]
            coupons = lg_split_split[2]
            issueDate = org_date
            refund_legs = refund_legs + s + '{' + f'"sequence":{i},"ticketNumber":"{ticketNumber}","coupons":"{coupons}","issueDate":"{issueDate}"' + '}'

            # RFT ORG_TICKET_NO
            # RFT Coupon_used
            if i == 1:
                org_ticket_no = ticketNumber
                Coupon_used = coupons

            i = i + 1

        refund_legs = refund_legs + ']'

    return [refund_legs, org_ticket_no, Coupon_used]

fc_fefund_legs = udf(fc_refund_legs, ArrayType(StringType()))
fc_fefund_legs_list = fc_fefund_legs("REF_LG","column_def","SAL2_split","org_ticket_no","Coupon_used")
bos_df_csv = bos_df_csv.withColumn('refund_legs', fc_fefund_legs_list[0])\
                         .withColumn('org_ticket_no', fc_fefund_legs_list[1])\
                         .withColumn('Coupon_used', fc_fefund_legs_list[2])

In [93]:
bos_df_csv2.select('refund_legs','org_ticket_no','Coupon_used').head(1)

['', 'REDACTED REF NAME', 'F', '-953.65', '0.00', '0.00', '', '-953.65', 'CA', 'CASH', '<\x83']
['125', '2161454822', '1234', '']
['', 'REDACTED REF NAME', 'F', '-953.65', '0.00', '0.00', '', '-953.65', 'CA', 'CASH', '<\x83']
['125', '2161454822', '1234', '']


[Row(refund_legs=',{"sequence":1,"ticketNumber":"1252161454822","coupons":"1234","issueDate":""}]', org_ticket_no='1252161454822', Coupon_used='1234')]

In [ ]:
ticketNumber = 'ticketNumber'
coupons = 'coupons'
issueDate = 'issueDate'
refund_legs = '[{' + f'"sequence":1,"ticketNumber":"{ticketNumber}","coupons":"{coupons}","issueDate":"{issueDate}"' + '}]'
print(refund_legs)


In [ ]:
# Data Validation
# def
def check_float(data, msg, header, length, decimal = 0):
    try:
        data_1 = data
        data = data if len(str(data).split('.')[0]) <= (length - decimal) else 'NULL'
        msg = msg if str(data) == str(data_1) else msg + ' # ' + header + 'OutOfRange: ' + str(data_1) + 'is out of range of (' +  str(length) + ',' + str(decimal) + ') # '
    except Exception as e:
        data = 'NULL'
        msg = msg + '# ERROR ' + str(e) + ' # '
    return data, msg


def fc_DV(attributes,original_fare, exchange_rate,fare_amount,tax_amount,total_amount):

    # attributes
    attributes = '' if attributes is None else attributes

    # original_fare
    # numeric(12, 5)
    original_fare, attributes = check_float(data = original_fare, msg = attributes , header = 'original_fare', length = 12, decimal = 5)

    # exchange_rate
    # numeric(12, 5)
    exchange_rate, attributes = check_float(data = exchange_rate, msg = attributes , header = 'exchange_rate', length = 12, decimal = 5)

    # fare_amount
    # numeric(12, 5)
    fare_amount, attributes = check_float(data = fare_amount, msg = attributes , header = 'fare_amount', length = 12, decimal = 5)

    # tax_amount
    # numeric(12, 5)
    tax_amount, attributes = check_float(data = tax_amount, msg = attributes , header = 'tax_amount', length = 12, decimal = 5)

    # total_amount
    # numeric(12, 5)
    total_amount, attributes = check_float(data = total_amount, msg = attributes , header = 'total_amount', length = 12, decimal = 5)

    return [attributes,original_fare,exchange_rate,fare_amount,tax_amount,total_amount]

fc_DV = udf(fc_DV, ArrayType(StringType()))
fc_DV_list = fc_DV('attributes', 'Original_Fare', 'exch_rate', 'Fare_amt', 'Tax_Amt', 'Total_amt')
bos_df_csv = bos_df_csv\
    .withColumn('attributes', fc_DV_list[0])\
    .withColumn('Original_Fare', fc_DV_list[1])\
    .withColumn('exch_rate', fc_DV_list[2])\
    .withColumn('Fare_amt', fc_DV_list[3])\
    .withColumn('Tax_Amt', fc_DV_list[4])\
    .withColumn('Total_amt', fc_DV_list[5])

In [16]:
def fc_DV_2(attributes, Agency_code,Source,ticket_type,Coupon_used):

    # attributes
    attributes = '' if attributes is None else attributes

    # Agency_code
    # numeric(10, 0)
    Agency_code, attributes = check_float(data = Agency_code, msg = attributes , header = 'Agency_code', length = 10, decimal = 0)

    # Source
    # numeric(10, 0)
    Source, attributes = check_float(data = Source, msg = attributes , header = 'Source', length = 10, decimal = 0)

    # ticket_type
    # numeric(10, 0)
    ticket_type, attributes = check_float(data = ticket_type, msg = attributes , header = 'ticket_type', length = 10, decimal = 0)


    # Coupon_used
    # numeric(10, 0)
    Coupon_used, attributes = check_float(data = Coupon_used, msg = attributes , header = 'Coupon_used', length = 10, decimal = 0)


    return [attributes,Agency_code,Source,ticket_type,Coupon_used]

fc_DV_2 = udf(fc_DV_2, ArrayType(StringType()))
fc_DV_2_list = fc_DV_2('attributes', 'Agency_code','Source','ticket_type','Coupon_used')
bos_df_csv = bos_df_csv\
    .withColumn('attributes', fc_DV_2_list[0])\
    .withColumn('Agency_code', fc_DV_2_list[1])\
    .withColumn('Source', fc_DV_2_list[2])\
    .withColumn('ticket_type', fc_DV_2_list[3])\
    .withColumn('Coupon_used', fc_DV_2_list[4])

In [17]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

sc = SparkContext.getOrCreate();
glueContext = GlueContext(sc)
spark = glueContext.spark_session

my_conn_options = {
    "dbtable": "flextravel.fx_trans_file",
    "database": "fts_cp_uat",
    "url": "jdbc:postgresql://flextravel-uat-serverless-pg.chzjoncadzav.ca-central-1.rds.amazonaws.com:5432/fts_cp_uat",
    "customJdbcDriverS3Path":"s3://flex-data-uat-canda-central/DEV/postgresql-42.6.0.jar",
    "customJdbcDriverClassName":"org.postgresql.Driver",
    "user":"flexuatuser",
    "password":"flexnewyearpwd@2022"
}

my_conn_options1 = {
    "dbtable": "flextravel.general_info",
    "database": "fts_cp_uat",
    "url": "jdbc:postgresql://flextravel-uat-serverless-pg.chzjoncadzav.ca-central-1.rds.amazonaws.com:5432/fts_cp_uat",
    "customJdbcDriverS3Path":"s3://flex-data-uat-canda-central/DEV/postgresql-42.6.0.jar",
    "customJdbcDriverClassName":"org.postgresql.Driver",
    "user":"flexuatuser",
    "password":"flexnewyearpwd@2022"
}

my_conn_options2 = {
    "dbtable": "public.fx_trans_bre_interim_bos_sprint2_test",
    "database": "fts_cp_uat",
    "url": "jdbc:postgresql://flextravel-uat-serverless-pg.chzjoncadzav.ca-central-1.rds.amazonaws.com:5432/fts_cp_uat",
    "customJdbcDriverS3Path":"s3://flex-data-uat-canda-central/DEV/postgresql-42.6.0.jar",
    "customJdbcDriverClassName":"org.postgresql.Driver",
    "user":"flexuatuser",
    "password":"flexnewyearpwd@2022"
}

df = glueContext.create_dynamic_frame.from_options(
    connection_type="postgresql",
    connection_options=my_conn_options,
    transformation_ctx="df",
)

df_GI = glueContext.create_dynamic_frame.from_options(
    connection_type="postgresql",
    connection_options=my_conn_options1,
    transformation_ctx="df",
)
GI_df = df_GI.toDF().select("id","tids_code")  



bos_df_csv=  bos_df_csv.withColumn( "Orginal_currency",F.when(length(col("Org_currency"))>5,'').otherwise(bos_df_csv.Org_currency))


bos_df_final = bos_df_csv.select("ticket_type","org_ticket_no","version_no","Filename","Transaction_Date","Agency_code","Source","Booking_channel","Ticket_No","PNR","Issue_date","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Legs","Original_Fare","Orginal_currency","exch_rate","Fare_amt","Tax_Amt","Total_amt","curr_code","FOP","Tax","Commission","Fare_Cons","attributes","file_path", "refund_legs")
                                                           
Bos_df_GI= bos_df_final.join(GI_df,
               bos_df_csv.Agency_code == GI_df.tids_code, 
               "left")  


Bos_df_GI=Bos_df_GI.withColumnRenamed("id","agent_id")

#Bos_df_GI.show(2, truncate=False)

# df1 = df.toDF().select("id", "supplier_id","sup_srv_map_id","file_name").where(df["file_name"] =='06.07_DCALLRECORDSAM')
# #df2=df1.withColumnRenamed("sup_srv_map_id","col0").withColumnRenamed("file_name","col1")
# # df1 = df.filter(f=lambda x: x["sup_srv_map_id"] in [65])

df1 = df.toDF()["id", "supplier_id","sup_srv_map_id","file_name"]
df1 = df1[df1["file_name"] =='06.07_DCALLRECORDSAM']


#df1.show()

Bos_df_file= Bos_df_GI.join(df1,
               bos_df_csv.Filename == df1.file_name, 
               "left")    

# Bos_df_file.show()

# FTS - 2598 ["Tax_Amt"].cast(IntegerType()) -> DoubleType, ["Total_amt"].cast(IntegerType()) -> DoubleType
Bos_df_file= Bos_df_file.withColumn("Original_Fare", Bos_df_file["Original_Fare"].cast(DoubleType())).withColumn("Fare_amt", Bos_df_file["Fare_amt"].cast(DoubleType())).withColumn("Tax_Amt", Bos_df_file["Tax_Amt"].cast(DoubleType()))\
            .withColumn("Total_amt", Bos_df_file["Total_amt"].cast(DoubleType())) .withColumn("exch_rate", Bos_df_file["exch_rate"].cast(DoubleType()))

Bos_df_file= Bos_df_file.withColumn("Commission_Amount", Bos_df_file["Fare_amt"].cast(DoubleType()))
                        
Bos_df_write = Bos_df_file.select("Transaction_Date","Agency_code","Source","Booking_channel","Ticket_No","PNR","Issue_date","Tour_code","Passenger_name","Passenger_Count","Coupon_used","Country_code","Legs","Original_Fare","Orginal_currency","exch_rate","Fare_amt","Tax_Amt","Total_amt","curr_code","FOP","Tax","Commission","Fare_Cons","agent_id","id","supplier_id","sup_srv_map_id","ticket_type","version_no","org_ticket_no","Commission_Amount","attributes","file_path", "refund_legs")


Bos_df_write = Bos_df_write.withColumnRenamed("Transaction_Date","transaction_date").withColumnRenamed("Agency_code","agency_code").withColumnRenamed("Source","source").withColumnRenamed("id","trans_file_id")\
                .withColumnRenamed("Booking_channel","booking_channel").withColumnRenamed("Ticket_No","ticket_number").withColumnRenamed("PNR","pnr").withColumnRenamed("Issue_date","issue_date").withColumnRenamed("Tour_code","tour_code")\
                .withColumnRenamed("Passenger_name","passenger_name").withColumnRenamed("Passenger_Count","passenger_count").withColumnRenamed("Coupon_used","coupon_used").withColumnRenamed("Country_code","country_code")\
                .withColumnRenamed("Legs","legs").withColumnRenamed("Original_Fare","original_fare").withColumnRenamed("Orginal_currency","original_currency").withColumnRenamed("exch_rate","exchange_rate").withColumnRenamed("Fare_amt","fare_amount")\
                .withColumnRenamed("Commission_Amount","commission_amount").withColumnRenamed("Tax_Amt","tax_amount").withColumnRenamed("Total_amt","total_amount").withColumnRenamed("curr_code","currency_code").withColumnRenamed("FOP","payment")\
                .withColumnRenamed("Tax","tax") .withColumnRenamed("Commission","commission").withColumnRenamed("Fare_Cons","fare_construction").withColumnRenamed("file_path","tax_on_commission")


In [18]:
Bos_df_write_dyn=DynamicFrame.fromDF(Bos_df_write,glueContext,'Bos_df_write_dyn')

df3=glueContext.write_dynamic_frame_from_options(
    frame=Bos_df_write_dyn,
    connection_type="postgresql",
    connection_options=my_conn_options2,
    transformation_ctx="dynamic_frame"
)
#df3.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File "/opt/amazon/spark/python/lib/pyspark.zip/pyspark/worker.py", line 450, in mapper
    result = tuple(f(*[a[o] for o in arg_offsets]) for (arg_offsets, f) in udfs)
  File "/opt/amazon/spark/python/lib/pyspark.z

In [1]:
# # mini batch test
# from pyspark.sql import SQLContext
# sqlContest = SQLContext(sc)

# i = 0
# k = 19000
# j = 100

# while (i+j < k):
#     print(i)

#     try:
#         pdk = Bos_df_write.toPandas()
#         pd1 = pdk.loc[i:i+j][:].fillna(0)
#         spark_df = sqlContest.createDataFrame(pd1)
        
#         Bos_df_write_dyn1=DynamicFrame.fromDF(spark_df,glueContext,'Bos_df_write_dyn')

#         df3=glueContext.write_dynamic_frame_from_options(
#             frame=Bos_df_write_dyn1,
#             connection_type="postgresql",
#             connection_options=my_conn_options2,
#             transformation_ctx="dynamic_frame"
#         )
#     except Exception as e:
#         print(e)
#     finally:
#         i = i + j

    



    


Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.3 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::251954244960:role/aws-glue-role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 94e43814-4481-46a2-bd42-9a375b67a951
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.3
--enable-glue-datacatalog true
Waiting for session 94e43814-4481-46a2-bd42-9a375b67a951 to get into ready status...
Session 94e43814-4481-46a2-bd42-9a375b67a951 has been created.



In [213]:
from datetime import datetime
print(type(datetime.now()) == '<class \'datetime.datetime\'>')

False


In [217]:
isinstance(datetime.now(), datetime)

True

In [218]:
issue_date = ''

In [219]:
issue_date[5]

IndexError: string index out of range